# Direct Radio Poll Test

In [ ]:
%autosave 0

In [ ]:
from __future__ import print_function
from builtins import *                  # python3 types
from time import sleep
import struct as pystruct
from binascii import hexlify
import sys
import os.path
import datetime

In [ ]:
sys.path.insert(0,"../tagnet")
from tagnet.tagmessages import TagMessage, TagPoll, TagGet, TagPut, TagDelete, TagHead
from tagnet.tagnames import TagName
from tagnet.tagtlv import TagTlv, TagTlvList, tlv_types

In [ ]:
sys.path.insert(0,"../tagfuse")
sys.path.insert(0,"../si446x/")
from tagfuse.radioutils import payload2values, radio_receive_msg, radio_send_msg
from tagfuse.radioutils import radio_start, radio_config, radio_show_config, radio_poll
from tagfuse.radioutils import radio_get_rtctime, radio_set_rtctime

In [ ]:
from tagfuse.tagfuseargs import process_cmd_args, get_cmd_args
import tagfuse.tagfuseargs
tagfuse.tagfuseargs.global_args
tagfuse.tagfuseargs.global_args=None
#sys.argv = ['tagfuse.py', '~/tags', '--disable_sparse']
sys.argv = ['tagfuse.py', '/home/pi/tag_store']
process_cmd_args()
get_cmd_args()
from tagfuse.TagFuseTree import TagFuseTagTree

In [ ]:
sys.path

In [ ]:
from tagfuse.tagfuseargs import __version__
'tagfuse', __version__

In [ ]:
print('Test Start Time: {}'.format(datetime.datetime.now()))
#print('Si446x Radio Device Driver Version: {}'.format(si446x_device_version()))

##  Start up Radio

In [ ]:
#radio = radio_start()
from si446x import Si446xRadio
from si446x import get_config_wds, get_name_wds, wds_default_config
radio=Si446xRadio(0)
if (radio == None):
    raise RuntimeError('radio_start: could not instantiate radio')
radio.unshutdown()
wds_default_config(0) # force alternate default config
radio.write_config()
radio.config_frr()

In [ ]:
get_name_wds()

In [ ]:
radio_show_config(radio.dump_radio())

### Set Defaults

In [ ]:
# default paramters
MAX_WAIT            = 1
MAX_RECV            = 2
MAX_PAYLOAD         = 254
MAX_RETRIES         = 3
RADIO_POWER         = 20
SHORT_DELAY         = 1

### Poll for Tags

In [ ]:
STOP


In [ ]:
radio_poll(radio)

In [ ]:
errors = 0
for i in range(20):
    poll_list = radio_poll(radio)
    if not poll_list:
        errors += 1
    #print(payload, rssi)
    if poll_list and (i % 2) == 0:
        for k,v in poll_list.iteritems():
            print(i, errors, k, v, '    ', end='\r')
print('errors', errors)

In [ ]:
import platform
platform.node()

In [ ]:
TagTlv(tlv_types)

In [ ]:
rssi, payload

In [ ]:
'send', sstatus, 'receive', rstatus

In [ ]:
radio.trace.display(radio.trace.filter(count=-10))

## Tag UTC Time

In [ ]:
STOP

In [ ]:
params = radio_get_rtctime(radio)
params[0]

In [ ]:
ut = datetime.datetime.utcnow()
radio_set_rtctime(radio, ut)
ut

## Listen for packets

In [ ]:
def show_name(name):
    name_str = ''
    for i in range(len(name)):
        tlv = name[i]
        if tlv.tlv_type() is tlv_types.NODE_ID:
            name_str += '/' + hexlify(name[i].value())
        else:
            name_str += '/' + name[i].value()
        
    return name_str

In [ ]:
def show_payload(payload):
    load_str = '({}) '.format(len(obj.payload))
    for tlv in payload:
        if tlv.tlv_type() is tlv_types.NODE_ID:
            load_str += 'node(' + hexlify(tlv.value()) + ')'
        else:
            load_str += ', ' + str(tlv.value())
    return load_str

In [ ]:
msgs = []
errors = 0
for i in range(200000):
    msg, rssi, status = radio_receive_msg(radio, MAX_RECV, .4)
    if msg:
        msgs.append((msg, rssi, datetime.datetime.now()))
    elif msgs:
        print('---')
        for msg, rssi, dtm in msgs:
            if len(msg) < 4:
                print('*** runt', len(msg), hexlify(msg))
            #info = '({}) {}, {}, {}. {}\n'.format(errors, datetime.datetime.now(), rssi, len(msg), hexlify(msg[:16]))
            obj = TagMessage(msg)
            #if obj and obj.payload:
                #info += '{}: {}\n'.format('Rsp' if obj.header.options.response else 'Req', show_name(obj.name))
                #info += '{}\n'.format(show_payload(obj.payload))
            #print(info)
            print('{} {} [r:{}, l:{}] {}: {}'.format(
                                                'Rsp' if obj.header.options.response else 'Req',
                                                dtm,
                                                rssi,
                                                len(msg),
                                                show_name(obj.name),
                                                show_payload(obj.payload) if obj and obj.payload else None,
                                                ))
        msgs = []

In [ ]:
hexlify(msg), len(msg)

In [ ]:
status

In [ ]:
status[-1]

In [ ]:
for k,v in rsp_obj.header.iteritems():
    print(k,v)

## Get Chip Status

In [ ]:
rsp_obj.header.options.response

In [ ]:
STOP

In [ ]:
print(radio.get_chip_status())